In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 14:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.4.2'

In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [9]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2019-10-01 00:11:29', dropOff_datetime='2019-10-01 00:13:22', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:11:43', dropOff_datetime='2019-10-01 00:37:20', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:56:29', dropOff_datetime='2019-10-01 00:57:47', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:23:09', dropOff_datetime='2019-10-01 00:28:27', PUlocationID='264', DOlocationID='264', SR_Flag=None, Affiliated_base_num

In [10]:
df = df.repartition(6)
df.write.parquet('fhv/2019/10/')

In [43]:
df = spark.read.parquet('fhv/2019/10')

In [44]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [45]:
## Q3

from pyspark.sql.functions import col, to_date, unix_timestamp, max

df = df.withColumn(
    'pickup_date',
    to_date(col('pickup_datetime')))
df.filter(df.pickup_date == '2019-10-15').count()

62610

In [48]:
## Q4 

df = df.withColumn('trip_length',
              ((unix_timestamp('dropOff_datetime') - unix_timestamp('pickup_datetime'))/3600
              ))
df.select(max(df.trip_length)).show()
# df.show()

+----------------+
|max(trip_length)|
+----------------+
|        631152.5|
+----------------+



In [51]:
## Q6

df_zone_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup_copy.csv')

df_zone_lookup = df_zone_lookup.repartition(6)
df_zone_lookup.write.parquet('taxi_zone_lookup/')
df_zone_lookup = spark.read.parquet('taxi_zone_lookup/')
df_zone_lookup.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [59]:
df_merged = df.join(df_zone_lookup, df.PUlocationID == df_zone_lookup.LocationID)
df_merged.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+-------------------+----------+-------+------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|        trip_length|LocationID|Borough|        Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+-------------------+----------+-------+------------+------------+
|              B01437|2019-10-18 16:58:08|2019-10-18 17:11:20|         264|         130|   null|                B01437| 2019-10-18|               0.22|       264|Unknown|          NV|         N/A|
|              B03048|2019-10-11 09:20:25|2019-10-11 09:37:15|         264|         264|   null|                B03048| 2019-10-11|0.28055555555555556|       264|Unknown|          NV|         N/A|
|              

In [60]:
df_merged.registerTempTable('trips_boroughs')

/home/USERNAME/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [68]:
spark.sql("""
SELECT COUNT(*) as cnt,
    Zone
FROM trips_boroughs 
GROUP BY Zone
ORDER BY cnt ASC
""").show()

+---+--------------------+
|cnt|                Zone|
+---+--------------------+
|  1|         Jamaica Bay|
|  2|Governor's Island...|
|  5| Green-Wood Cemetery|
|  8|       Broad Channel|
| 14|     Highbridge Park|
| 15|        Battery Park|
| 23|Saint Michaels Ce...|
| 25|Breezy Point/Fort...|
| 26|Marine Park/Floyd...|
| 29|        Astoria Park|
| 39|    Inwood Hill Park|
| 47|       Willets Point|
| 53|Forest Park/Highl...|
| 57|  Brooklyn Navy Yard|
| 62|        Crotona Park|
| 77|        Country Club|
| 89|     Freshkills Park|
| 98|       Prospect Park|
|105|     Columbia Street|
|110|  South Williamsburg|
+---+--------------------+
only showing top 20 rows

